In [ ]:
print("ok!")

ok!


In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import pinecone
# import pinecone 
# from pinecone import Pinecone
# from langchain_pinecone import PineconeVectorStore  # Make sure to install this package
from langchain_community.vectorstores import Pinecone as LangchainPinecone


from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [ ]:
PINECONE_API_KEY = "pcsk_2Utdae_AHvuS9XVfHBxGxKVW3R7scQenoL8Lb1B86bJNqSJBa2F2FiujnjyLhw7pPs3qJu"
PINECONE_API_ENV = "us-east-1"

In [ ]:
#Extract data from PDF

In [ ]:

def load_pdf(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents


In [ ]:
extracted_data = load_pdf("data/")

In [ ]:
# extracted_data

In [ ]:
#create text chunks
def text_split(extracted_data):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
  text_chunks = text_splitter.split_documents(extracted_data)

  return text_chunks

In [ ]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 5860


In [ ]:
# text_chunks

In [ ]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
embeddings = download_hugging_face_embeddings()

C:\Users\fouzia\AppData\Local\Temp\ipykernel_13592\1337643473.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\fouzia\anaconda3\envs\mchatbot\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
# query_result

In [ ]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader

# Import the correct Pinecone modules
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

# Define your Pinecone API key and index name
PINECONE_API_KEY = "pcsk_2Utdae_AHvuS9XVfHBxGxKVW3R7scQenoL8Lb1B86bJNqSJBa2F2FiujnjyLhw7pPs3qJu"
INDEX_NAME = "chatbot"

# Function to load PDF documents
def load_pdf(data_path):
    loader = DirectoryLoader(
        data_path,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents

# Function to split text into chunks
def text_split(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(documents)
    return text_chunks

# Function to download and initialize HuggingFace embeddings
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

# Main execution
if __name__ == "__main__":
    # Load PDF documents
    extracted_data = load_pdf("data/")
    print(f"Loaded {len(extracted_data)} documents")
    
    # Split text into chunks
    text_chunks = text_split(extracted_data)
    print(f"Created {len(text_chunks)} text chunks")
    
    # Initialize embeddings
    embeddings = download_hugging_face_embeddings()
    
    # Test embeddings
    test_query = "Hello world"
    query_result = embeddings.embed_query(test_query)
    print(f"Embedding dimension: {len(query_result)}")
    
    # Initialize Pinecone client
    pc = Pinecone(api_key=PINECONE_API_KEY)
    
    # Check if index exists, if not create it
    existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
    
    if INDEX_NAME not in existing_indexes:
        # Create index if it doesn't exist
        print(f"Creating new index: {INDEX_NAME}")
        pc.create_index(
            name=INDEX_NAME,
            dimension=len(query_result),  # Use the dimension from our test embedding
            metric="cosine",
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
            )
        )
    
    # Connect to the index
    index = pc.Index(INDEX_NAME)
    
    # Create vector store
    vector_store = PineconeVectorStore(
        index=index,
        embedding=embeddings,
        text_key="text"
    )
    
    # Prepare texts and metadata
    texts = [chunk.page_content for chunk in text_chunks]
    metadatas = [chunk.metadata for chunk in text_chunks]
    
    # Add texts to vector store
    vector_store.add_texts(texts=texts, metadatas=metadatas)
    
    print(f"Successfully added {len(texts)} chunks to Pinecone index '{INDEX_NAME}'")
    
    # Test a query
    query = "What is this document about?"
    results = vector_store.similarity_search(query, k=3)
    
    print(f"\nQuery: {query}")
    print("Top 3 results:")
    for i, doc in enumerate(results):
        print(f"{i+1}. {doc.page_content[:100]}...")

Loaded 637 documents
Created 5860 text chunks
Embedding dimension: 384
Successfully added 5860 chunks to Pinecone index 'chatbot'

Query: What is this document about?
Top 3 results:
1. er will be corrected in future editions.
This book is printed on recycled paper that meets Environme...
2. er will be corrected in future editions.
This book is printed on recycled paper that meets Environme...
3. Bejel
GEM -0433 to 0624 - B  10/22/03 6:08 PM  Page 460...


In [ ]:

import os
# If we already have an index we can load it like this
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import HuggingFaceEmbeddings

PINECONE_API_KEY = "pcsk_2Utdae_AHvuS9XVfHBxGxKVW3R7scQenoL8Lb1B86bJNqSJBa2F2FiujnjyLhw7pPs3qJu"
# <-- Replace this with your actual key
INDEX_NAME = "chatbot"  # <-- Replace with your actual index name

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Connect to Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)

# Create vector store
docsearch = PineconeVectorStore(
    index=index,
    embedding=embeddings,
    text_key="text"
)

# Perform query
query = "What are Allergies"
docs = docsearch.similarity_search(query, k=3)

# Pretty print results
for i, doc in enumerate(docs):
    print(f"\nResult {i+1}:\n{doc.page_content}\n")



Result 1:
Purpose
Allergy is a reaction of the immune system. Nor-
mally, the immune system responds to foreign microor-
ganisms and particles, like pollen or dust, by producing
specific proteins called antibodies that are capable of
binding to identifying molecules, or antigens, on the
foreign organisms. This reaction between antibody and
antigen sets off a series of reactions designed to protect
the body from infection. Sometimes, this same series of


Result 2:
Purpose
Allergy is a reaction of the immune system. Nor-
mally, the immune system responds to foreign microor-
ganisms and particles, like pollen or dust, by producing
specific proteins called antibodies that are capable of
binding to identifying molecules, or antigens, on the
foreign organisms. This reaction between antibody and
antigen sets off a series of reactions designed to protect
the body from infection. Sometimes, this same series of


Result 3:
reaction. Allergic rhinitis is characterized by an itchy,
runny nose, o

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """Use the following context to answer the question.
If you don't know the answer, just say you don't know.

Context:
{context}

Question:
{question}
"""

PROMPT = PromptTemplate(
    template=prompt_template, 
    input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}


In [ ]:
from langchain.llms import CTransformers

llm = CTransformers(
    model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
    model_type="llama",
    config={
        'max_new_tokens': 512,
        'temperature': 0.8
    }
)


In [ ]:
from langchain.chains import RetrievalQA

# Create the retrieval QA chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
# while True:
#     user_input = input("Input Prompt: ")
#     if user_input.lower() in ["exit", "quit"]:
#         break
#     result = qa({"query": user_input})
#     print("Response: ", result["result"])


C:\Users\fouzia\AppData\Local\Temp\ipykernel_1612\3253210397.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = qa({"query": user_input})


Response:  
A) is caused by a virus
B) can only be contracted through sexual contact
C) is transmitted through direct contact with an infected person's blood
D) can be spread through the air
E) occurs only in certain parts of the world


In [ ]:
print("ok!!")